In [1]:
!pip install geemap

In [2]:
from glob import glob
import os

import hydrofunctions as hf
import earthpy as et
import seaborn as sns
import pandas as pd
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import folium
import matplotcheck.notebook as nb
from matplotcheck.base import PlotTester
import matplotcheck.autograde as ag
import numpy as np


# Installs geemap package
import geemap.eefolium as geemap

# We are adding this for you as there is a bug in hydrofunctions that
# Turns off plotting
%matplotlib inline

# Prettier plotting with seaborn
sns.set(font_scale=1.5, style="whitegrid")

# Authenticates and initializes Earth Engine
import ee

In [3]:
ee.Authenticate()
ee.Initialize()  

Enter verification code: 4/1AX4XfWjTutotpuWHeLIeZ8oYmdOLAmfwszqJ7kIqeUXRWASTOinl7KRpdNA

Successfully saved authorization token.


In [4]:
Map = geemap.Map(center=[32.3,-90.9], zoom = 11)
Map

In [5]:
#roi = ee.Geometry.Point([32.3,-90.9])
roi = ee.Geometry.Point([-114.762293, 36.06462])

In [6]:
#set start date, end date, collect images from'COPERNICUS/S2_SR'

start_date = '2019-03-16'
end_date = '2019-05-17'
#cloud_threshold = 0.1
ndwi_threshold = 0

images = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(roi)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)
    
))

print(images.size().getInfo())
print(images.aggregate_array('system:id').getInfo())

16
['COPERNICUS/S2_SR/20190317T182109_20190317T182716_T11SPA', 'COPERNICUS/S2_SR/20190317T182109_20190317T182716_T11SPV', 'COPERNICUS/S2_SR/20190317T182109_20190317T182716_T11SQA', 'COPERNICUS/S2_SR/20190317T182109_20190317T182716_T11SQV', 'COPERNICUS/S2_SR/20190421T181921_20190421T183135_T11SQV', 'COPERNICUS/S2_SR/20190426T181929_20190426T183110_T11SPA', 'COPERNICUS/S2_SR/20190426T181929_20190426T183110_T11SPV', 'COPERNICUS/S2_SR/20190426T181929_20190426T183110_T11SQA', 'COPERNICUS/S2_SR/20190426T181929_20190426T183110_T11SQV', 'COPERNICUS/S2_SR/20190501T181921_20190501T183133_T11SPV', 'COPERNICUS/S2_SR/20190501T181921_20190501T183133_T11SQA', 'COPERNICUS/S2_SR/20190501T181921_20190501T183133_T11SQV', 'COPERNICUS/S2_SR/20190506T181929_20190506T182811_T11SPA', 'COPERNICUS/S2_SR/20190506T181929_20190506T182811_T11SPV', 'COPERNICUS/S2_SR/20190506T181929_20190506T182811_T11SQA', 'COPERNICUS/S2_SR/20190506T181929_20190506T182811_T11SQV']


In [7]:
dates = images.aggregate_array('system:time_start').map(
    lambda d: ee.Date(d).format('YYYY-MM-dd')
)
print(dates.getInfo())

['2019-03-17', '2019-03-17', '2019-03-17', '2019-03-17', '2019-04-21', '2019-04-26', '2019-04-26', '2019-04-26', '2019-04-26', '2019-05-01', '2019-05-01', '2019-05-01', '2019-05-06', '2019-05-06', '2019-05-06', '2019-05-06']


In [8]:
def extract_water(img):

    ndwi_image = img.normalizedDifference(['B3', 'B8'])
    water_image = ndwi_image.gt(ndwi_threshold)
    return water_image


In [9]:
ndwi_images = images.map(extract_water)

In [10]:
first_ndwi_image = ee.Image(ndwi_images.toList(ndwi_images.size()).get(0)).selfMask()
Map.addLayer(first_ndwi_image, {'palette': 'blue'}, 'First NDWI')
Map

In [11]:
def ras_to_vec(img):
    vec = img.selfMask().reduceToVectors(scale=30, maxPixels=60000000)
    vec = vec.filterBounds(roi)
    return vec.set({'area': vec.geometry().area(1).divide(1e6).round()})

In [12]:
ndwi_vectors = ndwi_images.map(ras_to_vec)
ndwi_vectors

In [13]:
areas = ndwi_vectors.aggregate_array('area')
print(areas.getInfo())

[41, 52, 279, 66, 52, 83, 84, 279, 66, 84, 76, 65, 83, 84, 76, 65]
